In [ ]:
from llama_cloud_services import LlamaParse

parser = LlamaParse(
    api_key="",
    language="bn",
    use_vendor_multimodal_model=True,
    vendor_multimodal_model_name="anthropic-sonnet-4.0"
)

result = parser.parse("../data/raw/HSC26-Bangla1st-Paper.pdf")

Started parsing the file under job_id ca1414c2-1aff-48fe-ab0d-8a660261873a
.

In [5]:
markdown_documents = result.get_markdown_documents(split_by_page=False)

In [8]:
markdown_documents[0].text_resource.text

'\n# 10 MINUTE SCHOOL\n\n## HSC 26 অনলাইন ব্যাচ\n### বাংলা • ইংরেজি • আইসিটি\n\n# বাংলা\n## ১ম পত্র\n\n### আলোচ্য বিষয়\n\n**অপরিচিতা**\n\nঅনলাইন ব্যাচ সম্পর্কিত যেকোনো জিজ্ঞাসায়,\n\n**কল করো** 📞 **16910**\n---\n\n\nHSC\'26\nঅনলাইন গাইড\nবাংলা • ইংরেজি • তথ্য ও যোগাযোগ প্রযুক্তি\n\n10 MINUTE SCHOOL\n\n## শিখনফল\n\n✓ নিমন্ত্রিত ব্যক্তির স্ট্যাটাস বিবেচনা করে ওঠার ফলে সমাজে পরিচয় সংকট সম্পর্কে ধারণা লাভ করবে।\n\n✓ তৎকালীন সমাজ-সভ্যতা ও মানবতার অবমাননা সম্পর্কে জানতে পারবে।\n\n✓ তৎকালীন সমাজের পশ্চাৎপদ কুসংস্কার সম্পর্কে জানতে পারবে।\n\n✓ তৎকালে সমাজে অন্ত্যজদের স্বতন্ত্র শিষ্টাচার সম্পর্কে অবগত হবে।\n\n✓ নারী কোমল ঠিক, কিন্তু দুর্বল নয়- কন্যাশিশু জীবনচরিত্র দ্বারা প্রতিষ্ঠিত এই সত্য অনুধাবন করতে পারবে।\n\n✓ মানুষ আশা নিয়ে বেঁচে থাকে- অপূর্বের দৃষ্টিতে মানবজীবনের এই চিরন্তন সত্যর্শন সম্পর্কে জ্ঞানলাভ করবে।\n\n## প্রাক-মূল্যায়ন\n\n১। অপূর্বের বাবা কী করে জীবিকা নির্বাহ করতেন?\nক) ডাক্তারি     খ) ওকালতি     গ) মাস্টারি     ঘ) ব্যবসা\n\n২। মামকে ভাগ্য দেবতার প্রধান এজেন্ট বলার কারণ, তার

In [ ]:
import re
import json
import os

from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import SentenceTransformerEmbeddings
from langchain_core.documents import Document


def clean_raw_markdown(raw_text: str) -> str:
    boilerplate_patterns = [
        r'HSC 26 অনলাইন ব্যাচ',
        r'HSC\'26 অনলাইন গাইড',
        r'HSC\'26 অনলাইন ক্যাট',
        r'HSC 26 অনলাইন স্কুল',
        r'বাংলা • ইংরেজি • আইসিটি',
        r'বাংলা-ইংরেজি-আইসিটি',
        r'বাংলা • ইংরেজি • তথ্য ও যোগাযোগ প্রযুক্তি',
        r'বাংলা-ইংরেজি-গণিত',
        r'10 MINUTE SCHOOL',
        r'অনলাইন ব্যাচ সম্পর্কিত যেকোনো জিজ্ঞাসায়,',
        r'কল করো 📞 16910',
        r'---',
        r'^\d+$'
    ]

    cleaned_text = raw_text
    for pattern in boilerplate_patterns:
        cleaned_text = re.sub(pattern, '', cleaned_text, flags=re.MULTILINE)

    correction_map = {
        'অনলাইন ক্যাট': 'অনলাইন ব্যাচ',
        'কন্যাগী': 'কল্যাণী',
        'শত্রুনাশ': 'শম্ভুনাথ',
        'শুনুরাখবারু': 'শম্ভুনাথবাবু',
        'শুভনাথ': 'শম্ভুনাথ'
    }
    for wrong, right in correction_map.items():
        cleaned_text = cleaned_text.replace(wrong, right)

    cleaned_text = re.sub(r'\n{3,}', '\n\n', cleaned_text)
    cleaned_text = '\n'.join([line.strip()
                             for line in cleaned_text.split('\n')])
    cleaned_text = cleaned_text.strip()

    return cleaned_text


def structure_and_chunk(cleaned_text: str) -> list[dict]:
    """
    Structures the cleaned text into semantic chunks with rich metadata.

    Args:
        cleaned_text: The cleaned markdown string.

    Returns:
        A list of dictionaries, where each dictionary represents a
        chunk with its content and metadata.
    """
    chunks = []
    sections = re.split(r'(?m)^#+\s*(.*?)$', cleaned_text)
    sections = sections[1:]

    for i in range(0, len(sections), 2):
        header = sections[i].strip()
        content = sections[i+1].strip()

        metadata_base = {
            "source": "HSC26-Bangla1st-Paper.pdf",
            "document_title": "অপরিচিতা",
            "author": "রবীন্দ্রনাথ ঠাকুর",
            "section": header
        }
        if header in ["মূল গল্প", "পাঠ পরিচিতি", "লেখক পরিচিতি"]:
            paragraphs = content.split('\n\n')
            for para_idx, paragraph in enumerate(paragraphs):
                if paragraph:  # Ensure not empty
                    chunk_meta = metadata_base.copy()
                    chunk_meta["chunk_type"] = "paragraph"
                    chunk_meta["chunk_id"] = f"{header.replace(' ', '_')}_{para_idx}"
                    chunks.append(
                        {"metadata": chunk_meta, "content": paragraph})
        elif header == "শব্দার্থ ও টীকা":
            lines = content.split('\n')
            for line_idx, line in enumerate(lines):
                if '|' in line and '---' not in line and 'মূল শব্দ' not in line:
                    chunk_meta = metadata_base.copy()
                    chunk_meta["chunk_type"] = "vocabulary_entry"
                    chunk_meta["chunk_id"] = f"vocab_{line_idx}"
                    chunks.append(
                        {"metadata": chunk_meta, "content": line.replace('|', ' | ')})
        elif "প্রশ্ন" in header:
            qa_blocks = re.split(r'(?m)^(\d+।)', content)
            qa_blocks = qa_blocks[1:]

            for q_idx in range(0, len(qa_blocks), 2):
                question_num = qa_blocks[q_idx].strip()
                qa_text = (question_num + " " + qa_blocks[q_idx+1]).strip()

                chunk_meta = metadata_base.copy()
                chunk_meta["chunk_type"] = "question_answer"
                chunk_meta["question_number"] = int(
                    re.search(r'\d+', question_num).group())

                if "বহুনির্বাচনী" in qa_text or re.search(r'\(ক\)', qa_text):
                    chunk_meta["question_style"] = "MCQ"
                elif "উদ্দীপকটি" in qa_text or "সৃজনশীল" in header:
                    chunk_meta["question_style"] = "Creative/Stimulus"
                else:
                    chunk_meta["question_style"] = "General"

                chunks.append({"metadata": chunk_meta, "content": qa_text})
        else:
            if content:
                chunk_meta = metadata_base.copy()
                chunk_meta["chunk_type"] = "full_section"
                chunks.append({"metadata": chunk_meta, "content": content})

    return chunks


if __name__ == "__main__":
    input_filename = "'n# 10 MINUTE SCHOOLnn## HSC 26 অনল.txt"
    output_filename = "cleaned_and_chunked_data.json"
    persist_directory = "../database/anwesha_chroma_manual"

    try:
        print("Step 1: Loading and cleaning raw markdown...")
        cleaned_markdown = clean_raw_markdown(
            markdown_documents[0].text_resource.text)
        print("Cleaning complete.\n")
        print("Step 2: Structuring and chunking data...")
        chunked_data = structure_and_chunk(cleaned_markdown)
        print(f"Chunking complete. Found {len(chunked_data)} chunks.\n")
        with open(output_filename, 'w', encoding='utf-8') as f:
            json.dump(chunked_data, f, ensure_ascii=False, indent=4)
        print(f"Intermediate chunked data saved to {output_filename}\n")
        print("Step 3: Converting chunks to LangChain Document objects...")
        splits = [Document(page_content=chunk['content'],
                           metadata=chunk['metadata']) for chunk in chunked_data]
        print(f"Created {len(splits)} LangChain Documents.\n")
        print("Step 4: Creating embeddings and loading into ChromaDB...")
        from langchain_huggingface import HuggingFaceEmbeddings

        embeddings = HuggingFaceEmbeddings(
            model_name="intfloat/multilingual-e5-large-instruct")
        os.makedirs(persist_directory, exist_ok=True)
        vectorstore = Chroma.from_documents(
            documents=splits,
            embedding=embeddings,
            persist_directory=persist_directory
        )
        print(f"Vector store created and persisted at: {persist_directory}\n")
        print("Step 5: Creating retriever and demonstrating usage...")
        retriever = vectorstore.as_retriever(
            search_kwargs={"k": 3})  # Retrieve top 3 results

        query = "অনুপমের মামার চরিত্র কেমন ছিল?"
        print(f"Executing sample query: '{query}'\n")

        retrieved_docs = retriever.invoke(query)

        print("--- Top Retrieved Documents ---")
        for i, doc in enumerate(retrieved_docs):
            print(f"\n--- Document {i+1} ---\n")
            print(f"Content: {doc.page_content}\n")
            print(
                f"Metadata: {json.dumps(doc.metadata, ensure_ascii=False, indent=2)}")
            print("-" * (len(f"--- Document {i+1} ---")))

    except FileNotFoundError:
        print(f"Error: The file '{input_filename}' was not found.")
        print("Please make sure the script is in the same directory as the text file to run this demonstration.")
    except Exception as e:
        print(f"An error occurred: {e}")

Step 1: Loading and cleaning raw markdown...
Cleaning complete.

Step 2: Structuring and chunking data...
Chunking complete. Found 235 chunks.

Intermediate chunked data saved to cleaned_and_chunked_data.json

Step 3: Converting chunks to LangChain Document objects...
Created 235 LangChain Documents.

Step 4: Creating embeddings and loading into ChromaDB...
Vector store created and persisted at: ../database/anwesha_chroma_manual

Step 5: Creating retriever and demonstrating usage...
Executing sample query: 'অনুপমের মামার চরিত্র কেমন ছিল?'

--- Top Retrieved Documents ---

--- Document 1 ---

Content: ৪৪।  উদ্দীপকের মমতা তোমার পঠিত কোন চরিত্রের প্রতিনিধিত্ব করে?

(ক) নাসি    (খ) সিপি    (গ) কল্যাণী    (ঘ) আম্মানি    উত্তর: গ

Metadata: {
  "question_style": "MCQ",
  "question_number": 44,
  "author": "রবীন্দ্রনাথ ঠাকুর",
  "section": "বোর্ড পরীক্ষার প্রশ্ন",
  "chunk_type": "question_answer",
  "source": "HSC26-Bangla1st-Paper.pdf",
  "document_title": "অপরিচিতা"
}
------------------

-

In [20]:
retrieved_docs = retriever.invoke("অনুপমের মামার চরিত্র কেমন ছিল?")

In [21]:
retrieved_docs

[Document(metadata={'section': 'বোর্ড পরীক্ষার প্রশ্ন', 'question_style': 'MCQ', 'question_number': 44, 'chunk_type': 'question_answer', 'author': 'রবীন্দ্রনাথ ঠাকুর', 'source': 'HSC26-Bangla1st-Paper.pdf', 'document_title': 'অপরিচিতা'}, page_content='৪৪।  উদ্দীপকের মমতা তোমার পঠিত কোন চরিত্রের প্রতিনিধিত্ব করে?\n\n(ক) নাসি    (খ) সিপি    (গ) কল্যাণী    (ঘ) আম্মানি    উত্তর: গ'),
 Document(metadata={'author': 'রবীন্দ্রনাথ ঠাকুর', 'chunk_type': 'question_answer', 'section': 'বোর্ড পরীক্ষার প্রশ্ন', 'question_number': 38, 'document_title': 'অপরিচিতা', 'question_style': 'MCQ', 'source': 'HSC26-Bangla1st-Paper.pdf'}, page_content="৩৮।  উদ্দীপকের শাফিকের 'অপরিচিতা' গল্পের কোন চরিত্রের প্রতিনিধি?\n\n(ক) অনুপম     (খ) হরিশ     (গ) বিনু     (ঘ) শম্ভুনাথ     উত্তর: খ"),
 Document(metadata={'source': 'HSC26-Bangla1st-Paper.pdf', 'chunk_id': 'মূল_গল্প_65', 'section': 'মূল গল্প', 'author': 'রবীন্দ্রনাথ ঠাকুর', 'document_title': 'অপরিচিতা', 'chunk_type': 'paragraph'}, page_content='পূর্বে বর খাইবে 

In [22]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="moonshotai/kimi-k2-instruct",
)

In [23]:
from langchain import hub

In [24]:
prompt = hub.pull("rlm/rag-prompt")

In [25]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [26]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [27]:
rag_chain.invoke("অনুপম তার মামার চেয়ে কত বছরের ছোট ছিল?")

'অনুপম তার মামার চেয়ে ঠিক কত বছরের ছোট ছিল, তথ্যটি প্রদত্ত প্রসঙ্গে উল্লেখ নেই; তাই উত্তর জানা সম্ভব নয়।'

In [28]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import ChatPromptTemplate

template = """You are an AI language model assistant. Your task is to generate five 
different versions of the given user question to retrieve relevant documents from a vector 
database. By generating multiple perspectives on the user question, your goal is to help
the user overcome some of the limitations of the distance-based similarity search. 
Provide these alternative questions separated by newlines. Original question: {question}"""
prompt_perspectives = ChatPromptTemplate.from_template(template)


generate_queries = (
    prompt_perspectives
    | ChatGroq(model="moonshotai/kimi-k2-instruct",)
    | StrOutputParser()
    | (lambda x: x.split("\n"))
)

In [29]:
from langchain.load import dumps, loads


def get_unique_union(documents: list[list]):
    flattened_docs = [dumps(doc) for sublist in documents for doc in sublist]
    unique_docs = list(set(flattened_docs))
    return [loads(doc) for doc in unique_docs]


question = "অনুপম তার মামার চেয়ে কত বছরের ছোট ছিল?"
retrieval_chain = generate_queries | retriever.map() | get_unique_union
docs = retrieval_chain.invoke({"question": question})
len(docs)

15

In [30]:
from operator import itemgetter
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough

template = """Answer the following question based on this context:

{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

llm = ChatGroq(
    model="moonshotai/kimi-k2-instruct",
)

final_rag_chain = (
    {"context": retrieval_chain,
     "question": itemgetter("question")}
    | prompt
    | llm
    | StrOutputParser()
)

final_rag_chain.invoke({"question": question})

'উত্তর: ১৮ বছর।\n\n(প্রসঙ্গত, ‘অপরিচিতা’ গল্পে অনুপমের বয়স ১৮ বছর এবং তার মামা তাঁর চেয়ে ১৮ বছরের বড়, সুতরাং অনুপম তার মামার চেয়ে ১৮ বছরের ছোট ছিল।)'

In [31]:
sample_queries = [
    "অপরিচিতা' গল্পে, অনুপমের মতে কে আসর জমাতে অদ্বিতীয়?",
    "অনুপম তার মামার চেয়ে কত বছরের ছোট ছিল?",
    "মন্দ নয় হে! খাঁটি সোনা বটে!' - এই উক্তিটি কার?",
    "কল্যাণীর বাবার নাম কী?",
    "বিবাহ-উপলক্ষ্যে কন্যাপক্ষকে কোথায় আসতে হয়েছিল?",
    "শম্ভুনাথ সেন পেশায় কী ছিলেন?",
    "অনুপম এবং তার মা কোন বাহনে তীর্থে যাচ্ছিলেন?",
    "রেলগাড়িতে কল্যাণীর সাথে কয়টি ছোট ছোট মেয়ে ছিল?",
    "বিবাহ ভাঙার পর কল্যাণী কী ব্রত গ্রহণ করে?",
    "গল্পের শেষে অনুপমের বয়স কত?"
]

expected_responses = [
    "হরিশ",
    "বছর ছয়েক",
    "বিনুদা",
    "শম্ভুনাথ সেন",
    "কলিকাতা",
    "ডাক্তার",
    "রেলগাড়ি",
    "দুটি-তিনটি",
    "মেয়েদের শিক্ষার ব্রত",
    "সাতাশ"
]

In [32]:
from ragas import EvaluationDataset


dataset = []

for query, reference in zip(sample_queries, expected_responses):
    relevant_docs = retriever.invoke(query)
    response = final_rag_chain.invoke({"question": query})
    dataset.append(
        {
            "user_input": query,
            "retrieved_contexts": [rdoc.page_content for rdoc in relevant_docs],
            "response": response,
            "reference": reference,
        }
    )

evaluation_dataset = EvaluationDataset.from_list(dataset)

APIStatusError: Error code: 413 - {'error': {'message': 'Request too large for model `moonshotai/kimi-k2-instruct` in organization `org_01k0a584zmftzsg16vm5awcxzj` service tier `on_demand` on tokens per minute (TPM): Limit 10000, Requested 18379, please reduce your message size and try again. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}